In [1]:
import fitz  # PyMuPDF
import re
import json
import datetime
import psycopg2
import base64

# Configuración de la base de datos
DB_NAME = "cda_busqueda"
DB_USER = "postgres"
DB_PASSWORD = "mysecretpassword"
DB_HOST = "postgres"
DB_PORT = "5432"

In [6]:
def extract_text_and_image(pdf_path):
    """
    Abre el PDF y extrae el texto de todas sus páginas.
    Además, recorre cada página para extraer la imagen de mayor tamaño (se asume que es la fotografía),
    y la codifica en base64 para poder insertarla en la BD.
    """
    doc = fitz.open(pdf_path)
    full_text = ""
    foto_base64 = None
    largest_area = 0

    for page in doc:
        # Extraer texto de la página
        text = page.get_text()
        full_text += text + "\n"

        # Extraer imágenes de la página
        images = page.get_images(full=True)
        for img in images:
            xref = img[0]
            pix = fitz.Pixmap(doc, xref)
            area = pix.width * pix.height
            if area > largest_area:
                largest_area = area
                # Si la imagen está en CMYK, convertir a RGB
                if pix.n >= 5:
                    pix = fitz.Pixmap(fitz.csRGB, pix)
                img_bytes = pix.tobytes("png")
                foto_base64 = base64.b64encode(img_bytes).decode('utf-8')
            pix = None

    return full_text, foto_base64

def parse_pdf_text(text):
    """
    Extrae la siguiente información utilizando expresiones regulares:
      - REPORTE NÚM.
      - FECHA DE ACTIVACIÓN
      - NOMBRE (se asume que es la primera línea "libre" luego de la línea de FECHA DE ACTIVACIÓN)
      - FECHA DE NACIMIENTO
      - EDAD
      - GÉNERO
      - FECHA DE LOS HECHOS
      - LUGAR DE LOS HECHOS
      - NACIONALIDAD
      - CABELLO
      - COLOR
      - COLOR DE OJOS
      - ESTATURA
      - PESO
      - SEÑAS PARTICULARES
      - RESUMEN DE LOS HECHOS

    Se utiliza una expresión para cada campo y se normalizan los espacios.
    """
    data = {}

    # REPORTE NÚM.
    match = re.search(r'REPORTE NÚM\.\s*:\s*([\w\d]+)', text)
    data['reporte_num'] = match.group(1).strip() if match else None

    # FECHA DE ACTIVACIÓN
    match = re.search(r'FECHA DE ACTIVACIÓN\s*:\s*([0-9/]+)', text)
    data['fecha_activacion'] = match.group(1).strip() if match else None

    # NOMBRE: se asume que es la primera línea sin etiqueta luego de "FECHA DE ACTIVACIÓN"
    lines = text.splitlines()
    nombre = None
    for i, line in enumerate(lines):
        if "FECHA DE ACTIVACIÓN" in line:
            for j in range(i + 1, len(lines)):
                candidate = lines[j].strip()
                if candidate and not re.search(
                    r'^(REPORTE NÚM\.|FECHA DE ACTIVACIÓN|FECHA DE NACIMIENTO|EDAD|GÉNERO|FECHA DE LOS HECHOS|LUGAR DE LOS HECHOS|NACIONALIDAD|CABELLO|COLOR\s*:|COLOR DE OJOS|ESTATURA|PESO|SEÑAS PARTICULARES|RESUMEN)',
                    candidate, re.IGNORECASE
                ):
                    nombre = candidate
                    break
            break
    data['nombre'] = nombre

    # FECHA DE NACIMIENTO
    match = re.search(r'FECHA DE NACIMIENTO\s*:\s*([0-9/]+)', text)
    data['fecha_nacimiento'] = match.group(1).strip() if match else None

    # EDAD (número, opcionalmente seguido de "años")
    match = re.search(r'EDAD\s*:\s*(\d+)', text, re.IGNORECASE)
    data['edad'] = match.group(1).strip() if match else None

    # GÉNERO
    match = re.search(r'GÉNERO\s*:\s*([\w]+)', text)
    data['genero'] = match.group(1).strip() if match else None

    # FECHA DE LOS HECHOS
    match = re.search(r'FECHA DE LOS\s+HECHOS\s*:\s*([0-9/]+)', text, re.IGNORECASE)
    data['fecha_hechos'] = match.group(1).strip() if match else None

    # LUGAR DE LOS HECHOS
    match = re.search(r'LUGAR DE LOS\s+HECHOS\s*:\s*([\w\s,]+)', text, re.IGNORECASE)
    data['lugar_hechos'] = match.group(1).strip() if match else None

    # NACIONALIDAD
    match = re.search(r'NACIONALIDAD\s*:\s*([\w]+)', text)
    data['nacionalidad'] = match.group(1).strip() if match else None

    # CABELLO
    match = re.search(r'CABELLO\s*:\s*([\w]+)', text)
    data['cabello'] = match.group(1).strip() if match else None

    # COLOR
    match = re.search(r'COLOR\s*:\s*([\w]+)', text)
    data['color'] = match.group(1).strip() if match else None

    # COLOR DE OJOS
    match = re.search(r'COLOR DE OJOS\s*:\s*([\w_]+)', text)
    data['color_ojos'] = match.group(1).strip() if match else None

    # ESTATURA (ejemplo: "1.6 m")
    match = re.search(r'ESTATURA\s*:\s*([\d\.]+\s*m)', text, re.IGNORECASE)
    data['estatura'] = match.group(1).strip() if match else None

    # PESO (ejemplo: "60 kg")
    match = re.search(r'PESO\s*:\s*([\d]+\s*kg)', text, re.IGNORECASE)
    data['peso'] = match.group(1).strip() if match else None

    # SEÑAS PARTICULARES: se captura hasta "RESUMEN DE HECHOS:" o similar
    match = re.search(r'SEÑAS PARTICULARES\s*:\s*(.*?)(?:RESUMEN)', text, re.DOTALL | re.IGNORECASE)
    if match:
        senas = " ".join(match.group(1).split())
        data['señas_particulares'] = senas
    else:
        data['señas_particulares'] = None

    # RESUMEN DE LOS HECHOS (o RESUMEN DE HECHOS)
    # Se captura hasta que se encuentre "LADA" o se llegue al final del texto.
    match = re.search(r'RESUMEN (?:DE LOS|DE) HECHOS\s*:\s*(.*?)(?:LADA|$)', text, re.DOTALL | re.IGNORECASE)
    if match:
        resumen = " ".join(match.group(1).split())
        data['resumen_hechos'] = resumen
    else:
        data['resumen_hechos'] = None

    return data

def insert_into_db(data, url_origen):
    """
    Inserta el registro en la tabla "desaparecidos".
    La columna 'datos' almacena en formato JSON toda la información extraída.
    """
    extraction_date = datetime.date.today()
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
        cur = conn.cursor()
        insert_query = """
            INSERT INTO public.desaparecidos (fecha_extraccion, url_origen, datos)
            VALUES (%s, %s, %s)
        """
        cur.execute(insert_query, (extraction_date, url_origen, json.dumps(data)))
        conn.commit()
        print("✅ Datos insertados correctamente en la base de datos.")
    except Exception as e:
        print(f"❌ Error al insertar en la BD: {e}")
    finally:
        cur.close()
        conn.close()

In [7]:
def main():
    pdf_path = "CreaAlertaPDFPublico.pdf"  # Ruta del PDF a procesar
    url_origen = "https://alertaamber.fgr.org.mx/"  # URL de origen (parametrizable)
    
    # Extraer texto e imagen (foto en base64) del PDF
    text, foto_base64 = extract_text_and_image(pdf_path)
    if not text:
        print("❌ No se pudo extraer el texto del PDF.")
        return
    
    # Extraer los datos del texto
    data = parse_pdf_text(text)
    # Insertar la foto extraída en el campo correspondiente
    data['foto'] = foto_base64

    print("Datos extraídos:")
    print(json.dumps(data, indent=4, ensure_ascii=False))

if __name__ == '__main__':
    main()


Datos extraídos:
{
    "reporte_num": "AAMX2056",
    "fecha_activacion": "07/03/2025",
    "nombre": "KEVIN ANTONIO  PEÑA  OLVERA",
    "fecha_nacimiento": "24/04/2008",
    "edad": "16",
    "genero": "Masculino",
    "fecha_hechos": "07/03/2025",
    "lugar_hechos": "GUADALAJARA, JALISCO\nNACIONALIDAD",
    "nacionalidad": "MEXICANA",
    "cabello": "Ondulado",
    "color": "Negro",
    "color_ojos": "Castaño_Obscuros",
    "estatura": "1.6 m",
    "peso": "60 kg",
    "señas_particulares": "CICATRIZ EN LA FRENTE DE ACNÉ, VERRUGA AL LADO DERECHO DE LA BOCA.",
    "resumen_hechos": "EL 15 DE FEBRERO DE 2025, EL ADOLESCENTE KEVIN ANTONIO PEÑA OLVERA, FUE VISTO POR ÚLTIMA VEZ EN GUADALAJARA, JALISCO. SIN QUE HASTA EL MOMENTO SE TENGA NOTICIAS DE SU PARADERO. SE CONSIDERA QUE LA INTEGRIDAD DEL ADOLESCENTE SE ENCUENTRA EN RIESGO TODA VEZ QUE PUEDE SER VÍCTIMA DE LA COMISIÓN DE UN DELITO.",
    "foto": "..."
}
